### Generate 500 data points X1, X2, . . . , X500 ∈ R^10 such that they are i.i.d.∼ N (0, I10). Decide Yi as follows:
### Yi =(+1 if ||Xi||^2 ≥ 9.34,−1 otherwise.)
### Use a two-terminal classification tree to classify the data, and apply boosting on top of the classification tree. Compute the error each time after applying the boosting algorithm. Plot the training error as a function of the number of iterations the boosting algorithm was applied.

In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


cov = np.identity(10)
mean = np.full(10,0)
x = np.random.multivariate_normal(mean, cov, 500)
y = np.full(500 , 1)
i = 0
for i in range  (0 ,500) :
    if np.dot(x[i], x[i]) < 9.34 :
        y[i] = y[i] * -1

In [ ]:
class DecisionStump:
    def __init__(self):
        self.feature = None
        self.threshold = None
        self.polarity = None

    def fit(self, X, y, weights):
        n_samples, n_features = X.shape
        best_error = np.inf

        # loop over all features and thresholds to find the best split
        for feature_idx in range(n_features):
            thresholds = np.unique(X[:, feature_idx])

            for threshold in thresholds:
                # try to split the data based on the threshold
                p = 1 if np.mean(y[X[:, feature_idx] < threshold]) > 0.5 else -1
                predictions = np.ones(n_samples) * -p
                predictions[X[:, feature_idx] >= threshold] = p

                # compute the weighted error of the split
                error = np.sum(weights[predictions != y])

                # update the best split found so far
                if error < best_error:
                    best_error = error
                    self.feature = feature_idx
                    self.threshold = threshold
                    self.polarity = p

    def predict(self, X):
        n_samples = X.shape[0]
        predictions = np.ones(n_samples) * -self.polarity
        predictions[X[:, self.feature] >= self.threshold] = self.polarity
        return predictions
class AdaBoost:
    def __init__(self, n_estimators=100):
        self.n_estimators = n_estimators
        self.estimators = []
        self.weights = []

    def fit(self, X, y):
        n_samples = X.shape[0]
        self.weights = np.ones(n_samples) / n_samples

        for i in range(self.n_estimators):
            stump = DecisionStump()
            stump.fit(X, y, self.weights)

            # update the weights based on the error of the stump
            error = np.sum(self.weights[stump.predict(X) != y])
            alpha = 0.5 * np.log((1 - error) / error)
            self.weights *= np.exp(-alpha * y * stump.predict(X))
            self.weights /= np.sum(self.weights)

            self.estimators.append((stump, alpha))

    def predict(self, X):
        n_samples = X.shape[0]
        predictions = np.zeros(n_samples)

        for stump, alpha in self.estimators:
            predictions += alpha * stump.predict(X)

        return np.sign(predictions)

def plot_training_error(estimators, X, y):
    n_samples = X.shape[0]
    train_error = np.zeros(estimators)

    for i in range(estimators):
        boost = AdaBoost(n_estimators=i+1)
        boost.fit(X, y)
        y_pred = boost.predict(X)
        train_error[i] = np.sum(y_pred != y) / n_samples

    plt.plot(train_error)
    plt.xlabel('Number of estimators')
    plt.ylabel('Training error')
    plt.show()


plot_training_error(100, x, y)
